Just for fun.

Running a larger than is sensible example in the compositional vtreat exporter (in this case it makes more sense to use the update exporter as described [here](https://github.com/WinVector/pyvtreat/blob/main/Examples/Database/update_joins.ipynb)).

In [1]:
import datetime
import os
import sys
import numpy as np
import pandas as pd

from data_algebra.data_ops import *
import data_algebra.SQLite
import data_algebra.test_util
import vtreat
from vtreat.vtreat_db_adapter import as_data_algebra_pipeline

sys.setrecursionlimit(30000)

In [2]:
# larger version of tests/test_db_adapter.py:test_db_adapter_monster()
def mk_example(n_rows:int = 100, n_vars:int = 50):
    step = 1/np.sqrt(n_vars)
    cols = dict()
    y = np.random.normal(size=n_rows)
    for i in range(n_vars):
        vname = f'v_{i}'
        v = np.random.choice(['a', 'b'], replace=True, size=n_rows)
        y = y + np.where(v == 'a', step, -step)
        cols[vname] = v
    vars = list(cols.keys())
    vars.sort()
    cols['y'] = y
    d = pd.DataFrame(cols)

    outcome_name = "y"
    cols_to_copy = [outcome_name]
    columns = vars + cols_to_copy

    treatment = vtreat.NumericOutcomeTreatment(
        cols_to_copy=cols_to_copy,
        outcome_name=outcome_name,
        params=vtreat.vtreat_parameters(
            {"sparse_indicators": False, "filter_to_recommended": False,}
        ),
    )
    d_train_treated = treatment.fit_transform(d)
    transform_as_data = treatment.description_matrix()
    source_descr = TableDescription(
        table_name='d_app',
        column_names=columns,
    )
    return {
        'transform_as_data': transform_as_data,
        'source_descr': source_descr,
    }

ex = mk_example(n_vars=500)
source_descr = ex['source_descr']
transform_as_data = ex['transform_as_data']


In [3]:
print(datetime.datetime.now())
ops = as_data_algebra_pipeline(
    source=source_descr,
    vtreat_descr=transform_as_data,
    treatment_table_name='transform_as_data',
)
print(datetime.datetime.now())

2021-12-16 10:27:57.488079
2021-12-16 10:28:04.934592


In [4]:
print(datetime.datetime.now())
ops_str = str(ops)
print(len(ops_str))
del ops_str
print(datetime.datetime.now())

2021-12-16 10:28:04.938398
1189598
2021-12-16 10:28:16.759611


In [5]:
print(datetime.datetime.now())
db_model = data_algebra.SQLite.SQLiteModel()
sql_str = db_model.to_sql(ops)
print(len(sql_str))
del sql_str
print(datetime.datetime.now())

2021-12-16 10:28:16.764426
62734179
2021-12-16 10:33:22.756291


The point being: 62 million character SQL query is probably larger than any reasonable query size limit. Again, in this case we don't need to make this query as we can use the [update exporter](https://github.com/WinVector/pyvtreat/blob/main/Examples/Database/update_joins.ipynb) at this scale.

In [6]:
1 + 1  # show we are done

2